In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/ColabNotebooks/')

In [3]:
%pip install torchmetrics

In [4]:
import torch
from torchmetrics.classification import BinaryF1Score, BinaryAccuracy, BinarySpecificity, BinaryPrecision, BinaryRecall
from model.wide_res_net import WideResNet
from model.smooth_cross_entropy import smooth_crossentropy
from coronary import Coronary
from utility.log import Log
from utility.initialize import initialize
from utility.step_lr import StepLR
from optimizers_impl.sam.sam import SAM
from utility.bypass_bn import enable_running_stats, disable_running_stats
import torch.nn as nn
from optimizers_impl.gsam import CosineScheduler


In [5]:
def train(model, device, train_loader, optimizer, log, scheduler):
    model.train()
    log.train(len_dataset=len(dataset.train))

    for batch in train_loader:
        inputs, targets = (b.to(device) for b in batch)
        inputs = inputs.permute(0, 3, 2, 1)
        enable_running_stats(model)
        predictions = model(inputs)
        loss = smooth_crossentropy(predictions, targets, smoothing=0.001)
        loss.mean().backward()
        optimizer.first_step(zero_grad=True)

        disable_running_stats(model)
        smooth_crossentropy(model(inputs), targets, smoothing=0.001).mean().backward()
        optimizer.second_step(zero_grad=True)

        with torch.no_grad():
            correct = torch.argmax(predictions.data, 1) == targets
            log(model, loss.cpu(), correct.cpu(), scheduler.lr())
            scheduler(epoch)
            # scheduler.step()


In [6]:
def test(model, device, test_loader, log):
    model.eval()
    log.eval(len_dataset=len(test_loader))

    with torch.no_grad():
        for batch in test_loader:
            inputs, targets = (b.to(device) for b in batch)
            inputs = inputs.permute(0, 3, 2, 1)

            predictions = model(inputs)
            loss = smooth_crossentropy(predictions, targets, smoothing=0.001)
            correct = torch.argmax(predictions, 1) == targets
            log(model, loss.cpu(), correct.cpu())


In [7]:
if __name__ == "__main__":

    num_epochs = 50
    model_name_path = '/content/drive/MyDrive/ColabNotebooks/results_logs/'
    model_name = 'mobilenet_v2-sam-75'
    model_name = model_name_path + model_name
    initialize(seed=111)
    log = Log(log_each=1, log_name=model_name)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
    model.classifier[1] = nn.Linear(1280, 2)
    model.to(device)
    model.eval()

    dataset = Coronary(batch_size=100, threads=2, img_size_definition=75)

    base_optimizer = torch.optim.SGD
    optimizer = SAM(model.parameters(), base_optimizer, rho=0.5, adaptive=True, momentum=0.8, lr=0.005, weight_decay=0.3)

    scheduler = StepLR(optimizer, 0.0004, num_epochs)

    for epoch in range(num_epochs):
        train(model, device, dataset.train, optimizer, log, scheduler)
        test(model, device, dataset.valid, log)
# ┃          49  ┃      0.1759  │      0.94 %  ┃   2.400e-05  │       00:06  ┃      0.4141  │     82.00 %  ┃

    log.flush()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


┏━━━━━━━━━━━━━━┳━━━━━━━╸T╺╸R╺╸A╺╸I╺╸N╺━━━━━━━┳━━━━━━━╸S╺╸T╺╸A╺╸T╺╸S╺━━━━━━━┳━━━━━━━╸V╺╸A╺╸L╺╸I╺╸D╺━━━━━━━┓
┃              ┃              ╷              ┃              ╷              ┃              ╷              ┃
┃       epoch  ┃        loss  │    accuracy  ┃        l.r.  │     elapsed  ┃        loss  │    accuracy  ┃
┠──────────────╂──────────────┼──────────────╂──────────────┼──────────────╂──────────────┼──────────────┨


/content/drive/MyDrive/ColabNotebooks/coronary.py:82: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  train_tensor_x = torch.Tensor(train_xs)


┃           0  ┃      0.6303  │      0.67 %  ┃   4.000e-04  │       00:04  ┃      0.6564  │     63.00 %  ┃
┃           1  ┃      0.5146  │      0.77 %  ┃   4.000e-04  │       00:01  ┃      0.6090  │     68.00 %  ┃
┃           2  ┃      0.4201  │      0.85 %  ┃   4.000e-04  │       00:01  ┃      0.5379  │     73.67 %  ┃
┃           3  ┃      0.3773  │      0.86 %  ┃   4.000e-04  │       00:01  ┃      0.4858  │     79.00 %  ┃
┃           4  ┃      0.3185  │      0.89 %  ┃   4.000e-04  │       00:01  ┃      0.4460  │     81.00 %  ┃
┃           5  ┃      0.2942  │      0.90 %  ┃   4.000e-04  │       00:01  ┃      0.4124  │     82.33 %  ┃
┃           6  ┃      0.2775  │      0.89 %  ┃   4.000e-04  │       00:01  ┃      0.4030  │     83.67 %  ┃
┃           7  ┃      0.2456  │      0.92 %  ┃   4.000e-04  │       00:01  ┃      0.4008  │     84.33 %  ┃
┃           8  ┃      0.2299  │      0.92 %  ┃   4.000e-04  │       00:01  ┃      0.4172  │     80.00 %  ┃
┃           9  ┃      0.2195  │      

In [8]:
metricF1 = BinaryF1Score().to(device)
metricAcc = BinaryAccuracy().to(device)
metricSpecificity = BinarySpecificity().to(device)
metricPrecision = BinaryPrecision().to(device)
metricRecall = BinaryRecall().to(device)

In [9]:
inputs = dataset.test_x.permute(0, 3, 2, 1).to(device)
predictions = model(inputs)
f1 = metricF1(torch.argmax(predictions, 1), dataset.test_y.to(device))
acc = metricAcc(torch.argmax(predictions, 1), dataset.test_y.to(device))
specificity = metricSpecificity(torch.argmax(predictions, 1), dataset.test_y.to(device))
precision = metricPrecision(torch.argmax(predictions, 1), dataset.test_y.to(device))
recall = metricRecall(torch.argmax(predictions, 1), dataset.test_y.to(device))

with open(model_name+'.txt', 'a') as file:
                file.write(f"\n \n F1: {f1}\n Accuracy: {acc}\n Specificity:{specificity}\n Precision: {precision}\n Recall: {recall}")

print(f1)
print(acc)
print(precision)
print(specificity)
print(recall)

tensor(0.9216, device='cuda:0')
tensor(0.9200, device='cuda:0')
tensor(0.9038, device='cuda:0')
tensor(0.9000, device='cuda:0')
tensor(0.9400, device='cuda:0')
